# Part 1 - find safe readings

NOTE: The input have varying number of readings per row.

In [17]:
import pandas as pd

df = pd.read_csv('input.txt', header=None, sep='\s+', names=range(8))
df.fillna(-1, inplace=True)
print(df.head())

    0   1   2   3   4     5     6     7
0   5   6   7  10  13  16.0  13.0  -1.0
1  19  21  24  27  28  28.0  -1.0  -1.0
2  16  18  20  21  23  25.0  29.0  -1.0
3  44  46  48  49  52  55.0  56.0  62.0
4  51  52  53  50  52  -1.0  -1.0  -1.0


In [90]:
def is_safe_reading(row):
    global failures_from_incr_decr
    global failures_from_delta
    last_reading = -1
    increasing = None
    for i in range(len(row)):
        if row[i] == -1:
            return True
        if last_reading == -1:
            last_reading = row[i]
            continue
        elif increasing is None:
            if row[i] > last_reading:
                increasing = True
            else:
                increasing = False
        if increasing and row[i] < last_reading:
            failures_from_incr_decr += 1
            return False
        elif not increasing and row[i] > last_reading:
            failures_from_incr_decr += 1
            return False
        
        delta = abs(row[i] - last_reading)
        last_reading = row[i]
        
        if delta > 3 or delta < 1:
            failures_from_delta += 1
            return False
        
    return True



In [70]:
if 'filter_result' in df.columns:
    df.drop('filter_result', axis=1, inplace=True)
df['filter_result'] = df.apply(is_safe_reading, axis=1)

In [33]:
df.head()

,0,1,2,3,4,5,6,7,filter_result
0,5,6,7,10,13,16.0,13.0,-1.0,False
1,19,21,24,27,28,28.0,-1.0,-1.0,False
2,16,18,20,21,23,25.0,29.0,-1.0,False
3,44,46,48,49,52,55.0,56.0,62.0,False
4,51,52,53,50,52,-1.0,-1.0,-1.0,False


In [71]:
len(df[df['filter_result'] == True])

299

The above is showing that there are 0 safe readings. Test it again using raw python

In [51]:
readings = []
with open('input.txt', 'r') as f:
    for line in f:
        readings.append([int(x) for x in line.strip().split(' ')])

readings[:10]



[[5, 6, 7, 10, 13, 16, 13],
 [19, 21, 24, 27, 28, 28],
 [16, 18, 20, 21, 23, 25, 29],
 [44, 46, 48, 49, 52, 55, 56, 62],
 [51, 52, 53, 50, 52],
 [10, 11, 12, 14, 11, 10],
 [80, 83, 85, 86, 88, 85, 85],
 [89, 90, 88, 90, 94],
 [85, 86, 83, 85, 92],
 [31, 32, 32, 33, 36]]

In [96]:
def is_safe_reading_remove_one(row):
    is_safe = is_safe_reading(row)
    if not is_safe:
        for i in range(len(row)):
            new_row = row[:i] + row[i+1:]
            if is_safe_reading(new_row):
                return True
    return is_safe 

# Answer below for part 2

In [98]:
failures_from_delta = 0
failures_from_incr_decr = 0

len([is_safe_reading(reading) for reading in readings if is_safe_reading_remove_one(reading)])


364

# Tests

In [97]:
# Input in form of row, will should work without modification; works with dampener.
from dataclasses import dataclass


@dataclass
class TestInput:
    row: list[int]
    expected_result_unmodified: bool
    expected_result_with_dampener: bool

    def __init__(self, row, expected_result_unmodified, expected_result_with_dampener):
        self.row = row
        self.expected_result_unmodified = expected_result_unmodified
        self.expected_result_with_dampener = expected_result_with_dampener

test_rows = [
    TestInput([7, 6, 4, 2, 1], True, True),
    TestInput([1, 2, 7, 8, 9], False, False),
    TestInput([9, 7, 6, 2, 1], False, False),
    TestInput([1, 3, 2, 4, 5], False, True),
    TestInput([8, 6, 4, 4, 1], False, True),
    TestInput([1, 3, 6, 7, 9], True, True),
]

success = True
for test_row in test_rows:
    if is_safe_reading(test_row.row) != test_row.expected_result_unmodified:
        success = False
        print(f'Failed unmodified for row: {test_row}')
    if is_safe_reading_remove_one(test_row.row) != test_row.expected_result_with_dampener:
        success = False
        print(f'Failed dampenened for row: {test_row}')

assert success



In [92]:
is_safe_reading_remove_one([9, 7, 6, 2, 1])
is_safe_reading([9, 7, 6, 1])

False
False
